In [ ]:
!git clone https://github.com/SinaPourSoltani/Visuomotor-Behaviour-Cloning.git
!pip3 install -r "Visuomotor-Behaviour-Cloning/requirements.txt"

import os 
os.kill(os.getpid(), 9)

In [ ]:
import sys
import json
from datetime import datetime
from torchvision import transforms
sys.path.append('Visuomotor-Behaviour-Cloning')
from BehaviourCloningCNN import *

# Without shadows
#data_link = "https://github.com/SinaPourSoltani/Visuomotor-Behaviour-Cloning/releases/download/v0.2/data.zip"
#is_stereo = False

# With Shadows
#data_link = "https://github.com/SinaPourSoltani/Visuomotor-Behaviour-Cloning/releases/download/v0.3/data.zip"
#is_stereo = False

# Stereo Images
#data_link = "https://github.com/SinaPourSoltani/Visuomotor-Behaviour-Cloning/releases/download/v0.4/data.zip" 
#is_stereo = True

augmentations = transforms.Compose([
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.GaussianBlur(kernel_size=5),
    transforms.RandomAffine(degrees=2,translate=(0.02,0.02),interpolation=transforms.functional.InterpolationMode.InterpolationMode.BILINEAR, fill=(255,255,255)),
])
# augmentations = None

load_data(data_link)
train_loader, valid_loader, test_loader = get_data_loaders(*get_episodes(),transforms=augmentations, is_stereo=is_stereo, std_noise_poke_vec=None)
model = get_model(is_stereo=is_stereo)

train_loss =  []
test_loss = []

In [ ]:
model = freeze_backbone(model, is_stereo=is_stereo)
train_loss, tmp_train_acc, test_loss, tmp_test_acc = train(model,train_loader,valid_loader, lr=1e-4, max_epochs=15, patience=15, is_stereo=is_stereo)
plot_history(train_loss, tmp_train_acc, test_loss, tmp_test_acc)

In [ ]:
model = unfreeze_backbone(model, is_stereo=is_stereo)
tmp_train_loss, tmp_train_acc, tmp_test_loss, tmp_test_acc = train(model,train_loader,valid_loader, lr=1e-5, max_epochs=15, patience=15, is_stereo=is_stereo)

# save loss for later use
train_loss += tmp_train_loss
test_loss += tmp_test_loss

plot_history(tmp_train_loss, tmp_train_acc, tmp_test_loss, tmp_test_acc)

In [ ]:
time_stamp = datetime.now().strftime("%d-%H-%M")
filename = "ResNet18_" + time_stamp + ".pth"
torch.save(model.state_dict(), filename)

summary = {}
summary["Model name"] = filename; 
summary["Epochs"] = len(train_loss)
summary["Train loss"] = train_loss
summary["Test loss"] = test_loss
summary["Is stereo"] = is_stereo

summary_file_name = "summary_" + time_stamp
with open(summary_file_name, 'w') as outfile:
            json.dump(summary, outfile)